<a href="https://colab.research.google.com/github/ekraking/NYC-Airbnb-Analysis/blob/main/MLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# تعيين شكل الرسوم البيانية ليكون أكثر وضوحًا
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('talk')

In [3]:
# ---------------------------------------------------------------------------
# الخطوة الأولى: تحميل واستكشاف البيانات
# ---------------------------------------------------------------------------
print("--- 1. تحميل واستكشاف البيانات ---")
# تحميل ملف البيانات
df = pd.read_csv('data1.csv')

# عرض أول 5 صفوف من البيانات لفهم هيكلها
print("أول 5 صفوف من البيانات:")
print(df.head())

# عرض معلومات عامة عن البيانات مثل أنواع الأعمدة والقيم غير الفارغة
print("\nمعلومات حول البيانات:")
df.info()


--- 1. تحميل واستكشاف البيانات ---
أول 5 صفوف من البيانات:
     id                                              name  host_id  \
0  2539                Clean & quiet apt home by the park     2787   
1  2595                             Skylit Midtown Castle     2845   
2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
3  3831                   Cozy Entire Floor of Brownstone     4869   
4  5022  Entire Apt: Spacious Studio/Loft by central park     7192   

     host_name neighbourhood_group neighbourhood  latitude  longitude  \
0         John            Brooklyn    Kensington  40.64749  -73.97237   
1     Jennifer           Manhattan       Midtown  40.75362  -73.98377   
2    Elisabeth           Manhattan        Harlem  40.80902  -73.94190   
3  LisaRoxanne            Brooklyn  Clinton Hill  40.68514  -73.95976   
4        Laura           Manhattan   East Harlem  40.79851  -73.94399   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0    

In [4]:
# ---------------------------------------------------------------------------
# **** قسم جديد: تحليل البيانات الاستكشافي مع التصور البياني (EDA) ****
# ---------------------------------------------------------------------------
print("\n--- مرحلة تحليل البيانات الاستكشافي (EDA) ---")

# 1. توزيع الأسعار
# سنقوم بفلترة الأسعار الأعلى من 500 دولار لجعل الرسم البياني أوضح
plt.figure(figsize=(12, 7))
sns.histplot(df[df['price'] < 500]['price'], bins=50, kde=True)
plt.title('Rental Price Distribution (Less than $500)')
plt.xlabel('Price ($)')
plt.ylabel('Number')
# plt.show()
plt.savefig('price_distribution.png')
plt.close()
print("Price distribution chart saved: price_distribution.png")

# 2. عدد الإعلانات حسب المنطقة
plt.figure(figsize=(12, 7))
sns.countplot(data=df, y='neighbourhood_group', order=df['neighbourhood_group'].value_counts().index, palette='viridis')
plt.title('Number of Airbnb listings in each area')
plt.xlabel('Number of ads')
plt.ylabel('The area')
# plt.show()
plt.savefig('listings_per_neighbourhood_group.png')
plt.close()
print("Price distribution chart saved: listings_per_neighbourhood_group.png")

# 3. توزيع أنواع الغرف
plt.figure(figsize=(12, 7))
sns.countplot(data=df, x='room_type', order=df['room_type'].value_counts().index, palette='plasma')
plt.title('Room Type Distribution')
plt.xlabel('Room type')
plt.ylabel('Number')
# plt.show()
plt.savefig('room_type_distribution.png')
plt.close()
print("Price distribution chart saved: room_type_distribution.png")

# 4. مقارنة الأسعار حسب المنطقة
plt.figure(figsize=(12, 7))
sns.boxplot(data=df[df['price'] < 500], x='neighbourhood_group', y='price', palette='magma')
plt.title('Compare price distribution by region (for prices under $500)')
plt.xlabel('The area')
plt.ylabel('Price ($)')
# plt.show()
plt.savefig('price_vs_neighbourhood_group.png')
plt.close()
print("Price distribution chart saved: price_vs_neighbourhood_group.png")


--- مرحلة تحليل البيانات الاستكشافي (EDA) ---
Price distribution chart saved: price_distribution.png


/tmp/ipython-input-3008950143.py:20: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, y='neighbourhood_group', order=df['neighbourhood_group'].value_counts().index, palette='viridis')


Price distribution chart saved: listings_per_neighbourhood_group.png


/tmp/ipython-input-3008950143.py:31: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=df, x='room_type', order=df['room_type'].value_counts().index, palette='plasma')


Price distribution chart saved: room_type_distribution.png


/tmp/ipython-input-3008950143.py:42: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df[df['price'] < 500], x='neighbourhood_group', y='price', palette='magma')


Price distribution chart saved: price_vs_neighbourhood_group.png


In [5]:
# ---------------------------------------------------------------------------
# الخطوة الثانية: تجهيز البيانات لمرحلة التعلم الإشرافي
# ---------------------------------------------------------------------------
print("\n--- 2. تجهيز البيانات للتعلم الإشرافي ---")

# التعامل مع القيم المفقودة: العمود 'reviews_per_month' يحتوي على قيم فارغة
# سنفترض أن القيم الفارغة تعني عدم وجود تقييمات، لذا سنملأها بالصفر
df['reviews_per_month'].fillna(0, inplace=True)

# تحديد المتغيرات العددية والفئوية التي سنستخدمها
# الميزات العددية
numeric_features = ['latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
# الميزات الفئوية
categorical_features = ['neighbourhood_group']
# المتغير المستهدف
target = 'room_type'

# فصل الميزات عن المتغير المستهدف
X = df[numeric_features + categorical_features]
y = df[target]

# تقسيم البيانات إلى مجموعة تدريب ومجموعة اختبار
# 80% للتدريب و 20% للاختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"حجم بيانات التدريب: {X_train.shape[0]} صف")
print(f"حجم بيانات الاختبار: {X_test.shape[0]} صف")

# بناء أنابيب المعالجة (Pipelines) لتجهيز البيانات
# خطوة مهمة لتوحيد قياس المتغيرات الرقمية وتحويل المتغيرات الفئوية
# معالج المتغيرات العددية: يقوم بتوحيد قياسها (Scaling)
numeric_transformer = StandardScaler()
# معالج المتغيرات الفئوية: يقوم بتحويلها إلى أرقام (One-Hot Encoding)
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# دمج المعالجين في معالج واحد باستخدام ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


--- 2. تجهيز البيانات للتعلم الإشرافي ---
حجم بيانات التدريب: 39116 صف
حجم بيانات الاختبار: 9779 صف


/tmp/ipython-input-1848669991.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['reviews_per_month'].fillna(0, inplace=True)


In [6]:
# ---------------------------------------------------------------------------
# الخطوة الثالثة: بناء وتدريب وتقييم نماذج التعلم الإشرافي
# ---------------------------------------------------------------------------
print("\n--- 3. بناء وتقييم نماذج التعلم الإشرافي ---")

# تعريف النماذج التي سنستخدمها
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes (Gaussian)": GaussianNB()
}

# حلقة لتكرار عملية التدريب والتقييم لكل نموذج
for name, model in models.items():
    print(f"\n--- تقييم نموذج: {name} ---")

    # إنشاء خط عمل (Pipeline) يربط بين معالجة البيانات والنموذج
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    # تدريب النموذج
    pipeline.fit(X_train, y_train)

    # إجراء التنبؤات على بيانات الاختبار
    y_pred = pipeline.predict(X_test)

    # حساب وعرض مقاييس الأداء
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # عرض مصفوفة الارتباك (Confusion Matrix)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=pipeline.classes_, yticklabels=pipeline.classes_)
    plt.title(f'Confusion Matrix - {name}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.savefig(f'confusion_matrix_{name.replace(" ", "_")}.png')
    plt.close() # نغلق الشكل لتجنب عرضه مباشرة هنا



--- 3. بناء وتقييم نماذج التعلم الإشرافي ---

--- تقييم نموذج: Logistic Regression ---
Accuracy: 0.8008

Classification Report:


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                 precision    recall  f1-score   support

Entire home/apt       0.86      0.78      0.82      5082
   Private room       0.75      0.86      0.80      4465
    Shared room       0.00      0.00      0.00       232

       accuracy                           0.80      9779
      macro avg       0.54      0.55      0.54      9779
   weighted avg       0.79      0.80      0.79      9779


--- تقييم نموذج: K-Nearest Neighbors (KNN) ---
Accuracy: 0.7823

Classification Report:
                 precision    recall  f1-score   support

Entire home/apt       0.81      0.81      0.81      5082
   Private room       0.76      0.79      0.77      4465
    Shared room       0.70      0.21      0.32       232

       accuracy                           0.78      9779
      macro avg       0.75      0.60      0.63      9779
   weighted avg       0.78      0.78      0.78      9779


--- تقييم نموذج: Naive Bayes (Gaussian) ---
Accuracy: 0.5480

Classification Report:
                 prec

In [7]:
# ---------------------------------------------------------------------------
# الخطوة الرابعة: تجهيز البيانات وتطبيق خوارزمية K-Means (التعلم غير الإشرافي)
# ---------------------------------------------------------------------------
print("\n--- 4. تطبيق خوارزمية K-Means (التعلم غير الإشرافي) ---")

# اختيار الميزات لعملية التجميع: الموقع الجغرافي والسعر
# سنقوم بأخذ عينة صغيرة من البيانات لتسريع العملية وتسهيل التصور
kmeans_df = df[['latitude', 'longitude', 'price']].copy()
kmeans_df_sample = kmeans_df[kmeans_df['price'] < 500].sample(n=10000, random_state=42) # فلترة الأسعار المرتفعة جداً وأخذ عينة

# توحيد قياس البيانات (Scaling) لأن K-Means حساسة لمقاييس المتغيرات
scaler = StandardScaler()
kmeans_scaled = scaler.fit_transform(kmeans_df_sample)

# استخدام طريقة الكوع (Elbow Method) لتحديد العدد الأمثل للمجموعات (K)
sse = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(kmeans_scaled)
    sse.append(kmeans.inertia_)

# رسم بياني لطريقة الكوع
plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Sum of Squared Errors (SSE)')
plt.xticks(k_range)
plt.savefig('kmeans_elbow_method.png')
plt.close()

# من الرسم البياني، سنختار العدد الأمثل للمجموعات (عادةً ما يكون عند "الكوع")
# لنفترض أن K=4 هو الخيار الأفضل
optimal_k = 4
print(f"\nتم اختيار العدد الأمثل للمجموعات K = {optimal_k}")

# تطبيق K-Means باستخدام العدد الأمثل للمجموعات
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans.fit(kmeans_scaled)

# إضافة labels المجموعات إلى بيانات العينة
kmeans_df_sample['cluster'] = kmeans.labels_



--- 4. تطبيق خوارزمية K-Means (التعلم غير الإشرافي) ---

تم اختيار العدد الأمثل للمجموعات K = 4


In [8]:
# ---------------------------------------------------------------------------
# الخطوة الخامسة: تصور نتائج K-Means
# ---------------------------------------------------------------------------
print("\n--- 5. تصور نتائج K-Means ---")
# رسم المجموعات على خريطة مبعثرة (Scatter Plot)
plt.figure(figsize=(12, 10))
sns.scatterplot(data=kmeans_df_sample, x='longitude', y='latitude', hue='cluster', palette='viridis', s=50, alpha=0.7)
plt.title('NYC Airbnb Listings Clusters (K-Means)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(title='Cluster')
plt.savefig('kmeans_clusters.png')
plt.close()

print("\n--- انتهى التنفيذ بنجاح! ---")
print("تم حفظ جميع الرسوم البيانية كملفات صور في مجلد العمل.")


--- 5. تصور نتائج K-Means ---

--- انتهى التنفيذ بنجاح! ---
تم حفظ جميع الرسوم البيانية كملفات صور في مجلد العمل.
